# eda

In [ ]:
import matplotlib.pyplot as plt
import glob
import numpy as np
import skimage.io as io
import json
import matplotlib.patches as patches
import cv2

In [ ]:
all_images = glob.glob("/root/data/lice-data/crops/blom-kjeppevikholmen/*/*/*.jpg")
print(len(all_images))

In [ ]:
for i in range(10):
    # pick a random image
    image_path = np.random.choice(all_images)
    random_image = io.imread(image_path)
    random_json = json.load(open(image_path.replace("jpg", "json")))
    im_height, im_width, _ = random_image.shape

    new_shape = (256, 768, 3)
    # new_shape = random_image.shape
    xratio = new_shape[0] / im_height
    yratio = new_shape[1] / im_width
#     print(xratio, yratio)

    f ,ax = plt.subplots(1, figsize=(20, 10))
    reshaped = cv2.resize(random_image, (new_shape[1] ,new_shape[0]), interpolation=cv2.INTER_LINEAR)
    ax.imshow(reshaped)
    for detection in random_json["detections"]:
        y, x, width, height = detection["position"]["left"], detection["position"]["top"], detection["position"]["width"], detection["position"]["height"]
        new_x = x*xratio
        new_y = y*yratio
        new_width = width*yratio
        new_height = height*xratio
        rec = patches.Rectangle((new_y, new_x),
                                new_width, 
                                new_height,
                                facecolor="none",
                                edgecolor="r",
                                linewidth=2)
        ax.add_patch(rec)
    plt.axis("off")
    plt.show()
    break

In [ ]:
random_json

In [ ]:
reshaped.shape

In [ ]:
rec.get_xy()

In [ ]:
import json

In [ ]:
# # open all json
# heights = []
# widths = []
# for img_path in all_images:
#     json_path = img_path.replace("jpg", "json")
#     detections = json.load(open(json_path))
#     for detection in detections["detections"]:
#         heights.append(detection["position"]["height"])
#         widths.append(detection["position"]["width"])

In [ ]:
plt.hist(heights)
plt.show()

In [ ]:
plt.hist(widths)
plt.show()

# experiment with unet

### load generator

In [ ]:
import random
import glob
import numpy as np
import skimage.io as io
import cv2
import json
import matplotlib.pyplot as plt
random.seed(148)

In [ ]:
all_images = glob.glob("/root/data/lice-data/crops/blom-kjeppevikholmen/*/*/*.jpg")
print(len(all_images))
random.shuffle(all_images)
cutoff = int(0.8*len(all_images))

In [ ]:
train_images = all_images[:cutoff]
val_images = all_images[cutoff:]
input_shape = [256, 768, 3]
batch_size = 8

In [ ]:
def get_heatmap(random_json, crop_shape, new_shape):
    xratio = new_shape[0] / crop_shape[0]
    yratio = new_shape[1] / crop_shape[1]
    
    heatmap = np.zeros((new_shape[0], new_shape[1]))
    for detection in random_json["detections"]:
        y, x, width, height = detection["position"]["left"], detection["position"]["top"], detection["position"]["width"], detection["position"]["height"]
        new_x = int(x*xratio)
        new_y = int(y*yratio)
        new_width = int(width*yratio)
        new_height = int(height*xratio)
        
        new_center_x = new_x + int(new_height / 2.0)
        new_center_y = new_y + int(new_width / 2.0)
        heatmap[new_x:new_x+new_height, new_y:new_y+new_width] = 0.8
        heatmap[new_center_x, new_center_y] = 1.0
        
    return np.expand_dims(heatmap, axis=-1)

In [ ]:
# heatmap = get_heatmap(random_json, random_image.shape, new_shape)

In [ ]:
def generator(images, BATCH_SIZE, input_shape):
    while True:
        x_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((BATCH_SIZE, input_shape[0], input_shape[1], 1))
        for i in range(BATCH_SIZE):
            random_path = np.random.choice(images)
            random_image = io.imread(random_path)
            random_json = json.load(open(random_path.replace("jpg", "json")))
            heatmap = get_heatmap(random_json, random_image.shape, input_shape)
            reshaped = cv2.resize(random_image, (input_shape[1] ,input_shape[0]), interpolation=cv2.INTER_LINEAR)
            x_batch[i, ...] = reshaped
            y_batch[i, ...] = heatmap
            
        yield x_batch, y_batch

In [ ]:
train_gen = generator(train_images, batch_size, input_shape)
val_gen = generator(val_images, batch_size, input_shape)

In [ ]:
X, Y = next(train_gen)

In [ ]:
for i in range(batch_size):
    plt.figure(figsize=(20, 10))
    plt.imshow(X[i, ...])
    plt.imshow(Y[i,...].squeeze(), alpha=0.5)
    plt.colorbar()
    plt.show()

### load model

In [ ]:
from unet import get_unet
import os
from keras.optimizers import Adam

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
model = get_unet(3, 256, 768, classes=1)

In [ ]:
lr=0.001
adam = Adam(lr=lr)
model.compile(adam, loss="binary_crossentropy", metrics=['binary_crossentropy'])

### training

In [ ]:
# # start training# start 
# history = model.fit_generator(
#         generator=train_gen,
#         steps_per_epoch=len(train_images) // batch_size,
#         epochs=10,
#         verbose=1,
#         validation_data= val_gen,
#         validation_steps= len(val_images) // batch_size)

In [ ]:
model.save("/root/data/models/lice/lice_heatmap.h5")

### test 

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model("/root/data/models/lice/lice_heatmap.h5")

In [ ]:
new_shape = (256, 768, 3)

In [ ]:
X, Y = next(val_gen)

In [ ]:
Y_pred = model.predict_on_batch(X)

In [ ]:
for i in range(batch_size):
    plt.figure(figsize=(20, 10))
    plt.imshow(X[i, ...])
    plt.imshow(Y_pred[i, ...].squeeze(), alpha=0.5)
    plt.axis('off')
    plt.show()
    plt.figure(figsize=(20, 10))
    plt.imshow(X[i, ...])
    plt.imshow(Y[i, ...].squeeze(), alpha=0.5)
    plt.show()
    print("#"*50)

### more viz full rez

In [ ]:
random_image = np.random.choice(val_images)
image = io.imread(random_image)
random_json = json.load(open(random_image.replace("jpg", "json")))
print(len(random_json["detections"]))
crop_shape = image.shape
xratio = new_shape[0] / crop_shape[0]
yratio = new_shape[1] / crop_shape[1]
X = cv2.resize(image, (new_shape[1] ,new_shape[0]), interpolation=cv2.INTER_LINEAR)
y_pred = model.predict_on_batch(np.expand_dims(X, axis=0))
y_pred_full = cv2.resize(y_pred.squeeze(), (crop_shape[1], crop_shape[0]), interpolation=cv2.INTER_LINEAR)

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(20, 20))
ax[0].imshow(image)
ax[0].axis("off")
ax[1].imshow(image)
ax[1].imshow(y_pred_full>0.1, alpha=0.5)
for detection in random_json["detections"]:
    y, x, width, height = detection["position"]["left"], detection["position"]["top"], detection["position"]["width"], detection["position"]["height"]
    rec = patches.Rectangle((y, x),
                            width, 
                            height,
                            facecolor="none",
                            edgecolor="r",
                            linewidth=2)
    ax[1].add_patch(rec)
# ax[1].axis("off")
# plt.colorbar()
plt.show()

In [ ]:
from skimage.measure import label
from scipy import ndimage

In [ ]:
test = y_pred_full>0.1
# kernel = np.ones((11, 11),np.uint8)
# test = cv2.erode(np.array(test, dtype=np.uint8), kernel)
labels = label(test)

In [ ]:
k = ndimage.label(test)

In [ ]:
tile = 128

In [ ]:
for i in range(k[1]):
    if i == 0:
        continue
    tmp = k[0] == i
    x, y = np.nonzero(tmp)
    centerx = int(np.mean(x))
    centery = int(np.mean(y))
    print(centerx, centery)
    lice_centered_crop = image[centerx-tile:centerx+tile, centery-tile:centery+tile, :]
    plt.imshow(lice_centered_crop)
    plt.show()
#     plt.imshow(cv2.resize(lice_centered_crop, (512, 512)))
#     plt.show()

In [ ]:
for lab in np.unique(labels):
    tmp = test == lab
    plt.imshow(tmp[0:300, 1000:2000])
    plt.show()

### recall

In [ ]:
from tqdm import tqdm

In [ ]:
ratios = []
for random_image in tqdm(val_images):
    # load the image and pred
    image = io.imread(random_image)
    random_json = json.load(open(random_image.replace("jpg", "json")))
    # print(len(random_json["detections"]))
    crop_shape = image.shape
    xratio = new_shape[0] / crop_shape[0]
    yratio = new_shape[1] / crop_shape[1]
    X = cv2.resize(image, (new_shape[1] ,new_shape[0]), interpolation=cv2.INTER_LINEAR)
    y_pred = model.predict_on_batch(np.expand_dims(X, axis=0))
    y_pred_full = cv2.resize(y_pred.squeeze(), (crop_shape[1], crop_shape[0]), interpolation=cv2.INTER_LINEAR)
    
    # compare detections 
    total_lice = len(random_json["detections"])
    detected = 0
    for detection in random_json["detections"]:
        y, x, width, height = detection["position"]["left"], detection["position"]["top"], detection["position"]["width"], detection["position"]["height"]
        if np.max(y_pred_full[x:x+height, y:y+width]) > 0.1:
            detected += 1
    ratios.append(detected / float(total_lice))

In [ ]:
np.mean(ratios)

In [ ]:
y_pred_full[y:y+width, x:x+height]

In [ ]:
x,y,width, height

In [ ]:
image.shape

In [ ]:
y_pred_full.shape